# Climatologies de l'atmosphère à partir de réanalyses météorologiques

Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021

L’objectif de ces activités Python est de mettre en évidence les grands traits de la circulation générale atmosphérique à travers l’étude de climatologies saisonnières des principaux paramètres atmosphériques : température, vent, pression, géopotentiel, humidité, précipitations. On exploitera pour cela des réanalyses météorologiques qui combinent des observations et le résultat de modèles numériques afin de constituer la meilleure connaissance possible de l’état de l'atmosphère, sur un domaine global, pour une date et une heure données. Les réanalyses utilisées ici sont celles du NCEP/NCAR (période 1948-présent, résolution horizontale de 2.5x2.5°, 17 niveaux pression sur la verticale).

Les fichiers de données au format netcdf (moyennes mensuelles NCEP/NCAR) doivent être téléchargés et placés dans le répertoire data :

Géopotentiel, température, vent zonal, vent méridien, vitesse verticale, humidité spécifique :
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/pressure/air.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/pressure/uwnd.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/pressure/vwnd.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/pressure/omega.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/pressure/shum.mon.mean.nc

Pression réduite au niveau de la mer, pression de surface, température à 2m, eau précipitable :
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/surface/slp.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/surface/pres.sfc.mon.mean
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/surface/pres.sfc.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/surface_gauss/air.2m.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Projects/Datasets/ncep.reanalysis.derived/surface/pr_wtr.eatm.mon.mean.nc

Précipitations :
- ftp://ftp.cdc.noaa.gov/Datasets/gpcp/precip.mon.mean.nc

Concepts Python illustrés :
- Exploitation de donnnées météorologiques au format netcdf (xarray)
- Calcul de climatologies saisonnières (xarray)
- Tracé de cartes (cartopy)
- Tracé de coupes et profils (matplotlib)
- Réalisation d'animation (PIL)

<div class="alert alert-warning">
<b>Instructions : </b>
<p><b>1) </b>Exécuter les cellules qui suivent de façon séquentielle</p>
<p><b>2) </b>Répondre aux questions (cadres de couleur jaune) dans les parties dédiées (cadres de couleur verte)</p>
<p><b>3) </b>Sauvegarder le notebook final au format pdf</p>
</div>

In [ ]:
# figures interactives dans le notebook
#%matplotlib notebook

# figures non interactives dans le notebook
%matplotlib inline

import os

import xarray as xr
import numpy as np
import netCDF4

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.path as mpath

from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

import warnings
warnings.filterwarnings('ignore')

# Traitement des données

In [ ]:
year1='1990'
year2='2019'

In [ ]:
diri="./data/"

# z, T, u, v, w, q
fz    = xr.open_dataset(diri+"hgt.mon.mean.nc").sel(time=slice(year1,year2))
ft    = xr.open_dataset(diri+"air.mon.mean.nc").sel(time=slice(year1,year2))+273.15
fu    = xr.open_dataset(diri+"uwnd.mon.mean.nc").sel(time=slice(year1,year2))
fv    = xr.open_dataset(diri+"vwnd.mon.mean.nc").sel(time=slice(year1,year2))
fw    = xr.open_dataset(diri+"omega.mon.mean.nc").sel(time=slice(year1,year2))
fq    = xr.open_dataset(diri+"shum.mon.mean.nc").sel(time=slice(year1,year2))/1000

# precip, MSLP, T2m, PW
fpr = xr.open_dataset(diri+"precip.mon.mean.nc").sel(time=slice(year1,year2))
fp    = xr.open_dataset(diri+"slp.mon.mean.nc").sel(time=slice(year1,year2))
fps    = xr.open_dataset(diri+"pres.sfc.mon.mean.nc").sel(time=slice(year1,year2))
ft2m  = xr.open_dataset(diri+"air2m.mon.mean.nc").sel(time=slice(year1,year2))
fpw   = xr.open_dataset(diri+"pr_wtr.mon.mean.nc").sel(time=slice(year1,year2))

# Température moyenne de la couche 1000-500hPa
ftavg    = xr.open_dataset(diri+"air.mon.mean.nc").sel(time=slice(year1,year2)).sel(level=slice(1000,500)).mean('level')+273.15

# Latitudes, niveaux verticaux
lat  = ft.lat.values
lat_t2m  = ft2m.lat.values
lat_pr  = fpr.lat.values
lat_pw  = fpw.lat.values
lat_ps  = fps.lat.values

lev = fz.level.values
levw = fw.level.values
levq = fq.level.values

print(ft)
print(fw)

In [ ]:
seasons=['DJF','JJA','MAM','SON']
months=['Jan','Feb','Mar','Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# z, T, u, v, w, q
fz_mean = fz.groupby('time.season').mean('time')
ft_mean = ft.groupby('time.season').mean('time')
fu_mean = fu.groupby('time.season').mean('time')
fv_mean = fv.groupby('time.season').mean('time')
fw_mean = fw.groupby('time.season').mean('time')
fq_mean = fq.groupby('time.season').mean('time')

# precip, MSLP, T2m, PW
fpr_mean = fpr.groupby('time.season').mean('time')
fps_mean = fps.groupby('time.season').mean('time')
fpr_mean_month = fpr.groupby('time.month').mean('time')

fp_mean = fp.groupby('time.season').mean('time')
ft2m_mean = ft2m.groupby('time.season').mean('time')
fpw_mean = fpw.groupby('time.season').mean('time')

# Température moyenne 1000-500hPa
ftavg_mean = ftavg.groupby('time.season').mean('time')

# Variables
z0 = fz_mean['hgt']
t0 = ft_mean['air']
u0 = fu_mean['uwnd']
v0 = fv_mean['vwnd']
w = fw_mean['omega']
q0 = fq_mean['shum']*1000
pr0 = fpr_mean['precip']
pr0_month = fpr_mean_month['precip']

p0 = fp_mean['slp']
pres0 = fps_mean['pres']/100
t2m0 = ft2m_mean['air']
pw0 = fpw_mean['pr_wtr']
tavg0 = ftavg_mean['air']

print(z0.shape) # saison,niveau,lat,lon
print(p0.shape) # saison,lat,lon
print(pr0_month.shape) # mois,lat,lon

In [ ]:
# Moyennes zonales (z, T, u, v, w, q, MSLP)
zz = z0.mean(axis=3)
tz = t0.mean(axis=3)
uz = u0.mean(axis=3)
vz = v0.mean(axis=3)
wz = w.mean(axis=3)
qz = q0.mean(axis=3)

pz = p0.mean(axis=2)
pz_annual = pz.mean(axis=0)

# Moyennes verticales 1000-100 hPa (T, u)
plev1=1000
plev2=100
iplev1 = list(lev).index(plev1)
iplev2 = list(lev).index(plev2)
t_vavg = tz[:,iplev1:iplev2,:].mean(axis=1)
t_vavg_annual = t_vavg.mean(axis=0)
u_vavg = uz[:,iplev1:iplev2,:].mean(axis=1)
u_vavg_annual = u_vavg.mean(axis=0)

print(zz.shape) # saison,niveau,lat
print(pz.shape) # saison,lat
print(u_vavg.shape) # saison,lat
print(pz_annual.shape) # lat
print(u_vavg_annual.shape) # lat

# Moyennes annuelles pour uz, vz wz
uz_annual=np.mean(uz, axis=0)
vz_annual=np.mean(vz, axis=0)
wz_annual=np.mean(wz, axis=0)

print(uz_annual.shape) # level,lat

In [ ]:
def lonflip(da):
    lon_name = 'lon'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[ lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
z=lonflip(z0)

uu=lonflip(u0)
vv=lonflip(v0)
t=lonflip(t0)
q=lonflip(q0)

tavg=lonflip(tavg0)

pr=lonflip(pr0)
pr_month=lonflip(pr0_month)

p=lonflip(p0)
pres=lonflip(pres0)
t2m=lonflip(t2m0)
pw=lonflip(pw0)

# Longitudes
lon  = z.lon.values
lon_ps  = pres.lon.values
lon_pr  = pr.lon.values
lon_pw  = pw.lon.values
lon_t2m  = t2m.lon.values
print(lon)

# Conversion de u et v en numpy array pour tracé des vecteurs vent (sinon bug)
u=np.array(uu)
v=np.array(vv)

In [ ]:
plevz1=1000
plevz2=500
ilev1 = list(lev).index(plevz1)
ilev2 = list(lev).index(plevz2)

z1 = z[:,ilev1,:,:]
z2 = z[:,ilev2,:,:]
dz = z2 - z1

print(np.min(dz))
print(np.max(dz))

# Fonctions graphiques

In [ ]:
# on s'arrête à 100hPa sur la varticale
def plot_zonal_mean(ax):
    ax.set_yscale('symlog')
    ax.set_ylim(1000, 100)
    ax.set_xticks(np.arange(-90, 120, 30))
    ax.set_yticks(np.arange(1000, 0, -100))
    ax.tick_params(labelsize=8)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlabel('Latitude')
    ax.set_ylabel('Pressure level')
    return ax

# on s'arrête à 10hPa sur la verticale
def plot_zonal_mean2(ax):
    ax.set_yscale('symlog')
    ax.set_ylim(1000, 10)
    ax.set_xticks(np.arange(-90, 120, 30))
    ax.set_yticks(np.arange(1000, 0, -100))
    ax.tick_params(labelsize=8)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlabel('Latitude')
    ax.set_ylabel('Pressure level')
    return ax

In [ ]:
#proj=ccrs.PlateCarree()
proj=ccrs.EqualEarth()

def plot_background(ax):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    return ax

In [ ]:
# cartes isobares z, t, u
levels_z = np.arange(10500,12800,100)
levels_t = np.arange(220,312,2)
levels_u = np.arange(-50,55,5)

# cartes épaisseurs et température moyenne
levels_dz = np.arange(4500,6050,50)
levels_tavg = np.arange(220,315,5)

# cartes en moyenne zonale z, t, u, w, v, q
levels_zz = np.arange(0,21000,1000)
levels_tz = np.arange(180,320,10)
levels_uz = np.arange(-50,52.5,2.5)
levels_uz_an = np.arange(0,55,5)
levels_wz = np.arange(-0.05,0.052,0.002)
levels_vz =[-7.0, -6.5, -6.0, -5.5, -5.0, -4.5, -4.0, -3.5, -3.0,
 -2.5, -2.0, -1.5, -1.0, -0.5, 0.5, 1.0, 1.5, 2.0, 2.5,
  3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7]
levels_qz = np.arange(0,20,1)

# cartes horizontales T2m, MSLP, précip, PW
levels_t2m = np.arange(200,322,2)
levels_p = np.arange(970,1042,2)
levels_ps = np.arange(500,1090,50)
levels_pr = np.arange(0,15,1)
levels_pw = np.arange(0,60,2.5)

# Etude du champ de température

La température caractérise l’agitation moléculaire moyenne. Elle est proportionnelle à l’énergie cinétique moyenne associée aux mouvements désordonnés des molécules dans la matière. L’unité du système international pour la température est le Kelvin (K). En météorologie on utilise très souvent le degré Celsius. Les échelles Kelvin et Celsius diffèrent uniquement par leur origine T(K) = T(°C) + 273,5.

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Temperature (K)')
#plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.xticks(np.arange(-90, 120, 30))

plt.title('Zonal mean temperature - '+str(plev1)+'-'+str(plev2)+ 'hPa average : NCEP '+year1+'-'+year2)

plt.plot(lat, t_vavg_annual, label='Annual')

for i in range(2):
    plt.plot(lat, t_vavg[i,:], label=seasons[i])
    plt.axvline(0, color='black', linestyle="--")
    
figname='./figs/tavg_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.legend()

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> En quoi le profil méridien de température est-il cohérent avec le bilan radiatif étudié précédemment ?</p>
<p><b>2) </b> Comparer les variations méridiennes de la température aux moyennes latitudes
 et dans les régions tropicales.</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

In [ ]:
lats_text=["10N","30N","45N","70N"]
ilat1 = list(lat).index(10)
ilat2 = list(lat).index(30)
ilat3 = list(lat).index(45)
ilat4 = list(lat).index(70)

fig = plt.figure(figsize=(15,8))
fig.suptitle('Temperature - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

plt.subplot(121)
plt.title('DJF')
plt.xlabel('Temperature (K)')
plt.ylabel('Pressure level (hPa)')
plt.yscale('log')
plt.gca().invert_yaxis()
plt.plot(tz[0,:,ilat1],lev, label=lats_text[0])
plt.plot(tz[0,:,ilat2],lev, label=lats_text[1])
plt.plot(tz[0,:,ilat3],lev, label=lats_text[2])
plt.plot(tz[0,:,ilat4],lev, label=lats_text[3])
plt.legend()

plt.subplot(122)
plt.title('JJA')
plt.xlabel('Temperature (K)')
plt.yscale('log')
plt.gca().invert_yaxis()
plt.plot(tz[1,:,ilat1],lev, label=lats_text[0])
plt.plot(tz[1,:,ilat2],lev, label=lats_text[1])
plt.plot(tz[1,:,ilat3],lev, label=lats_text[2])
plt.plot(tz[1,:,ilat4],lev, label=lats_text[3])
plt.legend()

figname='./figs/t_zmean_profiles'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> Commenter la variation verticale de la température dans la troposphère.</p>
<p><b>2) </b> Donner le niveau moyen et la température moyenne de la tropopause en zone tropicale, ainsi qu'aux latitudes moyennes et polaires.</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Temperature - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean2(ax)
    cf = ax.contourf(lat, lev, tz[i,:,:], levels_tz, cmap='jet', extend='both')
    c = ax.contour(lat, lev, tz[i,:,:], levels_tz, colors='black', linewidths=1)
    plt.clabel(c, levels_tz, fmt='%1.2i')
 
cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('mgp', size='small')

figname='./figs/t_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Qu'observe-t-on dans la stratosphère polaire de l'hémisphère d'hiver. Au-dessus de quel pôle cela est-il le plus marqué ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

In [ ]:
axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Temperature (K) at 2 meters : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(seasons[i], fontsize=14)
    cf = ax.contourf(lon_t2m, lat_t2m, t2m[i,:,:], levels_t2m, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    c = ax.contour(lon_t2m, lat_t2m, t2m[i,:,:], levels_t2m, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

figname='./figs/t2m_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Que constate-t-on sur les bords est des océans Pacifique et
 Atlantique ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

In [ ]:
plevt=850
ilev = list(lev).index(plevt)

axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Temperature (K) at '+str(plevt)+' hPa : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(seasons[i], fontsize=14)
    cf = ax.contourf(lon, lat, t[i,ilev,:,:], levels_t, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    c = ax.contour(lon, lat, t[i,ilev,:,:], levels_t, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

figname='./figs/t_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Sur les cartes isobares à 850 hPa, localiser les zones les plus chaudes de l’hémisphère d’été.</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Etude du champ de vent zonal

Le vent mesure la vitesse de déplacement de l’air par rapport à la Terre. Le vent zonal, noté u (unité : m/s) caractérise le déplacement parallèle à un cercle de latitude. Par convention, u > 0 correspond à un vent dirigé d’ouest en est.

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.xlabel('Latitude')
plt.ylabel('Zonal wind (m/s)')
#plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.xticks(np.arange(-90, 120, 30))

plt.title('Zonal mean zonal wind - '+str(plev1)+'-'+str(plev2)+ 'hPa average : '+year1+'-'+year2)

plt.plot(lat, u_vavg_annual, label='Annual')

for i in range(2):
    plt.plot(lat, u_vavg[i,:], label=seasons[i])
    plt.axhline(0, color='black', linestyle="--")
    plt.axvline(0, color='black', linestyle="--")
    
figname='./figs/uavg_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.legend()

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Comparer la direction et l’intensité des vents zonaux moyens dominants aux
 moyennes latitudes et dans les régions tropicales.</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Zonal wind - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean(ax)
    cf = ax.contourf(lat, lev, uz[i,:,:], levels_uz, cmap='jet', extend='both')
    c = ax.contour(lat, lev, uz[i,:,:], levels_uz, colors='black', linewidths=1)
    plt.clabel(c, levels_uz, fmt='%2.1i')
 
cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('m/s', size='small')

figname='./figs/u_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> Comparer la direction et l’intensité des vents zonaux moyens dominants aux moyennes latitudes et dans les régions tropicales.</p>
<p><b>2) </b> Quelle est l’intensité moyenne des maximums de vents d’est observés près du sol en zone tropicale (alizés). En quelle saison sont-ils les plus intenses ?</p>
<p><b>3) </b> En quelle saison et vers quel niveau et à quelle latitude observe-t-on un jet
 d’est de haute troposphère en zone tropicale (jet d’est tropical, JET) ?</p>
<p><b>4) </b> Vers quel niveau de pression sont situés les vents les plus forts aux latitudes subtropicales (jets d’ouest subtropicaux, JOST) ? En quelle saison sont-ils les plus intenses ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Zonal wind - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean2(ax)
    cf = ax.contourf(lat, lev, uz[i,:,:], levels_uz, cmap='jet', extend='both')
    c = ax.contour(lat, lev, uz[i,:,:], levels_uz, colors='black', linewidths=1)
    plt.clabel(c, levels_uz, fmt='%2.1i')
 
cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('m/s', size='small')

figname='./figs/u_zmean_climatology2'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Commenter les structures de vent zonal dans la stratosphère ainsi que leur variations saisonnières.
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

In [ ]:
plevz=200
plevu=200
ilev = list(lev).index(plevu)

axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Zonal wind (m/s) at '+str(plevu)+' hPa : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    ax.coastlines()
    ax.set_title(seasons[i], fontsize=14)
    plot_background(ax)
    cf = ax.contourf(lon, lat, u[i,ilev,:,:], levels_u, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    c = ax.contour(lon, lat, u[i,ilev,:,:], levels_u, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

figname='./figs/u_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Dans quel hémisphère la structure spatiale des noyaux JOST de haute troposphère est-elle plus irrégulière ? Proposer une explication.
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Etude des champs de vent méridien et de vitesse verticale

Le vent mesure la vitesse de déplacement de l’air par rapport à la Terre. Le vent méridien, noté v (unité : m/s) caractérise le déplacement parallèle à un méridien. Par convention, v > 0 correspond à un vent dirigé du sud vers le nord. La vitesse verticale des particules d’air dans l’atmosphère est un paramètre non mesuré par les instruments d’observation classiques mais accessible via les modèles numériques. On l’exprime en m/s si la coordonnée verticale est l’altitude. Les modèles numériques ayant une coordonnée verticale en pression on exprime le plus souvent la vitesse verticale (oméga, unité : Pa/s). Elle est négative lorsque l’air « monte » (ascendance) et positive lorsqu’il « descend » (subsidence).

Remarque : on se restreint ici à l’étude de la circulation méridienne et verticale en moyenne zonale entre les latitudes 60S-60N.

In [ ]:
fig = plt.figure(figsize=(15., 10.))
ax = fig.add_subplot(1, 1, 1)
fig.suptitle('Vertical velocity (Pa/s), meridional wind (m/s) and zonal wind (m/s) - zonal mean : NCEP '+year1+'-'+year2, fontsize=12)

ax.set_title('Annual mean', fontsize=8)
plot_zonal_mean(ax)
ax.set_xlim(-60, 60)
cf = ax.contourf(lat, levw, wz_annual[:,:], levels_wz, cmap='seismic', extend='both')
c = ax.contour(lat, lev, vz_annual[:,:], levels_vz, colors='black', linewidths=1)
plt.clabel(c, levels_vz, fmt='%1.1f')
c2 = ax.contour(lat, lev, uz_annual[:,:], levels_uz_an, colors='green', linewidths=0.5)
plt.clabel(c2, levels_uz_an, fmt='%2.1i')

cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.75, pad=0.15)
cb.set_label('Pa/s', size='small')

figname='./figs/wv_zmean_annual_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> Décrire les deux cellules fermées méridiennes et verticales que l’on observe en moyenne zonale et annuelle au voisinnage de l’équateur : localisation des zones d’ascendance, de subsidence, de convergence et de divergence Ces deux cellules méridiennes et verticales constituent la circulation de Hadley.</p>
<p><b>2) </b> D’après le signe de la composante méridienne du vent en zone tropicale, donner la direction des alizés de chaque hémisphère. Les alizés convergent-ils ou divergent-ils au niveau des branches ascendantes des cellules de Hadley ?</p>
<p><b>3) </b> En moyenne annuellen où sont localisées les branches subsidentes des cellules de Hadley par rapport aux jets d’ouest subtropicaux vus précédemment (représentés ici en vert) ?</p>   
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=2, figsize=(15, 8), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Vertical velocity (Pa/s) and meridional wind (m/s) - zonal mean : NCEP '+year1+'-'+year2, fontsize=12)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=8)
    plot_zonal_mean(ax)
    ax.set_xlim(-60, 60)
    cf = ax.contourf(lat, levw, wz[i,:,:], levels_wz, cmap='seismic', extend='both')
    c = ax.contour(lat, lev, vz[i,:,:], levels_vz, colors='black', linewidths=1)
    plt.clabel(c, levels_vz, fmt='%1.1f')

cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=1, pad=0)
cb.ax.tick_params(labelsize=8) 
cb.set_label('Pa/s', size='small')

figname='./figs/wv_zmean_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> Décrire le cycle saisonnier de la circulation de Hadley en termes d’intensité, d'extension méridienne et de position.</p>
<p><b>2) </b> A quelle période de l’année les cellules de Hadley semblent-elles le plus symétriques par rapport à l'équateur ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# Etude du champ de masse (pression)

Dans l’atmosphère, la pression en un point est pratiquement égale au poids d’une colonne d’air de section horizontale égale à 1 m2, située au-dessus du point de mesure, et s’étendant jusqu’au sommet de l’atmosphère. L’unité SI de la pression est le Pascal (Pa). 1 Pa = 1 N.m-2. Au niveau de la mer, P ~ 10 5Pa = 1000 hPa (1013,25 hPa en condition « standard »). Par habitude, certains météorologistes parlent encore en bars ou en millibars, avec 1 bar = 10 5Pa. 1 mbar = 1 hPa. Le relief influence très fortement les valeurs de la pression au sol et le paramètre « pression de surface » donne plus d’informations sur les différences d’altitude entre les différents points de mesure que sur les variations horizontales et temporelles de la pression. Ainsi, pour comparer les différentes mesures de pression entre elles, on estime à partir des données de pression, de température et d’humidité au sol, la pression qu’on aurait à un niveau de référence donné : le niveau moyen de la mer. On parle de pression réduite au niveau de la mer (Pmer, ou MSLP en anglais pour « Mean Sea Level Pressure »).

In [ ]:
fig = plt.figure(figsize=(15,10))

plt_title = 'Mean sea level pressure (hPa) - zonal mean : NCEP '+year1+'-'+year2
plt.xlabel('Latitude')
plt.ylabel('MSLP (hPa)')
#plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.xticks(np.arange(-90, 120, 30))

plt.title(plt_title)

plt.plot(lat, pz_annual, label='Annual')

for i in range(2):
 plt.plot(lat, pz[i,:], label=seasons[i])
 plt.axhline(1013, color='black', linestyle="--")
 plt.axvline(0, color='black', linestyle="--")

plt.legend()

figname='./figs/slp_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> En moyenne annuelle, à quelles latitudes observe-t-on maximums/minimums de la masse de la colonne d’air ?</p>
<p><b>2) </b> Comment ces structures varient-elles en fonction de la saison ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

In [ ]:
axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Mean Sea level pressure (hPa) : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(seasons[i], fontsize=14)
    cf = ax.contourf(lon, lat, p[i,:,:], levels_p, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    c = ax.contour(lon, lat, p[i,:,:], levels_p, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

figname='./figs/slp_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> Nommer les anticyclones océaniques subtropicaux. Au niveau de quelle branche de la cellule de Hadley sont-ils localisés ?</p>
<p><b>2) </b> Au niveau de anticyclones subtropicaux Atlantique et Pacifique, la pression est-elle plus forte sur les flancs est ou ouest des bassins ?</p>
<p><b>3) </b> Nommer les structures climatologiques présentes sur l’Atlantique nord en DJF.</p>
<p><b>4) </b> Selon la saison, localiser les principales dépressions thermiques continentales (dépressions au sol correspondant à de l'air chaud, cf. carte de température à 850 hPa).</p>
<p><b>5) </b> Selon la saison, localiser les principaux anticyclones thermiques continentaux (anticyclone au sol correspondant à de l'air froid, cf. carte de température à 850 hPa). Localiser l’anticyclone thermique le plus intense.</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# Etude du champ de masse (géopotentiel, altitude gépotentielle)

Le champ de géopotentiel φ représente une énergie potentielle de pesanteur : plus une particule d'air se situe haut dans l’atmosphère, plus son énergie potentielle et donc son géopotentiel sont élevés.

φ = g*z où g est l’accélération de pesanteur et z l’altitude (unité : J/kg)

Exemple : le travail à exercer contre la pesanteur terrestre de référence (g = 9,81 m/s²) pour élever une masse unitaire de 1 kg de la surface jusqu’à 5000 mètres sera de 9,81*5000 = 49050 J. Cette masse aura donc acquis un géopotentiel de 49 050 J/Kg, c’est-à-dire une énergie latente capable de se matérialiser si l’objet retombait à la surface (accélération lors de la chute). En divisant par l’accélération de pesanteur de référence, on trouve l’altitude géopotentielle Z (altitude géopotentielle est l’altitude à laquelle est atteint un potentiel de pesanteur donné), dans ce cas équivalente à l’altitude géométrique : Z= φ/g = 5000 mètres géopotentiels (mgp).

En météorologie , les isolignes d'égale altitude géopotentielle Z sont appelées les isohypses (courbes de niveau des surfaces isobares). Elles s’expriment en mgp (approximativement égal au mètre). On parle de hauts ou de bas géopotentiels pour désigner respectivement des maximums d’altitude d’une surface isobare (zone de hautes pressions en altitude) ou des minimums d’altitude d’une surface isobare (zone de basses pressions en altitude).

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Geopotential height - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean(ax)
    cf = ax.contourf(lat, lev, zz[i,:,:], levels_zz, cmap='jet', extend='both')
    c = ax.contour(lat, lev, zz[i,:,:], levels_zz, colors='black', linewidths=1)
    plt.clabel(c, levels_zz, fmt='%1.2i')
    ax.axhline(100, color='white', linestyle="--")
    ax.axhline(200, color='white', linestyle="--")
    ax.axhline(300, color='white', linestyle="--")
    ax.axhline(500, color='white', linestyle="--")
    ax.axhline(700, color='white', linestyle="--")
    ax.axhline(850, color='white', linestyle="--")
    ax.axhline(925, color='white', linestyle="--")
 
cf = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cf.set_label('mgp', size='small')

figname='./figs/z_zmean_climatology1'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> A 45N, quelles sont les altitudes géopotentielles moyennes des surfaces isobares 850 hPa, 700 hPa, 500 hPa, 300 hPa et 200 hPa (été et hiver) ?</p>
<p><b>2) </b> Même question à 10N.</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

<div class="alert alert-danger">
<b>Question : </b>
<p><b>1) </b> Retrouver les correspondances altitude-pression ci-dessus par le code à partir du tableau zz[season,lev,lat]</p>
</div>

In [ ]:
print('Tableau latitude')
print(lat)
print('Tableau level')
print(lev)
print('Taille tableau géopotentiel (season,lev,lat)')
print(zz.shape)

In [ ]:
lev_text=["925 hPa","850 hPa","700 hPa","500 hPa","300 hPa", "200 hPa","100 hPa"]
iplev1 = list(lev).index(925)
iplev2 = list(lev).index(850)
iplev3 = list(lev).index(700)
iplev4 = list(lev).index(500)
iplev5 = list(lev).index(300)
iplev6 = list(lev).index(200)
iplev7 = list(lev).index(100)

fig = plt.figure(figsize=(12,15))
fig.suptitle('Geopotential height - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

plt.subplot(211)
plt.title('DJF')
plt.xlabel('latitude')
plt.ylabel('height')
#plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.xticks(np.arange(-90, 120, 30))
plt.plot(lat,zz[0,iplev1], label=lev_text[0])
plt.plot(lat,zz[0,iplev2], label=lev_text[1])
plt.plot(lat,zz[0,iplev3], label=lev_text[2])
plt.plot(lat,zz[0,iplev4], label=lev_text[3])
plt.plot(lat,zz[0,iplev5], label=lev_text[4])
plt.plot(lat,zz[0,iplev6], label=lev_text[5])
plt.plot(lat,zz[0,iplev7], label=lev_text[6])
plt.legend()

plt.subplot(212)
plt.title('JJA')
plt.xlabel('latitude')
plt.ylabel('height')
#plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.xticks(np.arange(-90, 120, 30))
plt.plot(lat,zz[1,iplev1], label=lev_text[0])
plt.plot(lat,zz[1,iplev2], label=lev_text[1])
plt.plot(lat,zz[1,iplev3], label=lev_text[2])
plt.plot(lat,zz[1,iplev4], label=lev_text[3])
plt.plot(lat,zz[1,iplev5], label=lev_text[4])
plt.plot(lat,zz[1,iplev6], label=lev_text[5])
plt.plot(lat,zz[1,iplev7], label=lev_text[6])
plt.legend()

figname='./figs/z_zmean_climatology2'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Dans quelles régions peut-on dire que les surfaces isobares sont quasi-horizontales ? Dans quelles régions les surfaces isobares sont-elles inclinées (i.e pente on observe une pente des surfaces isobares) ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

In [ ]:
plevz=200
ilev = list(lev).index(plevz)

axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Geopotential height (mgp) at '+str(plevz)+' hPa : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(seasons[i], fontsize=14)
    cf = ax.contourf(lon, lat, z[i,ilev,:,:], levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    c = ax.contour(lon, lat, z[i,ilev,:,:], levels_z, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

figname='./figs/z_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> A quelles latitudes et en quelle saison la pente de la surface isobare à 200 hPa est-elle la plus importante ?</p>
<p><b>2) </b> Dans quelles régions et en quelle saison les dissymétries zonales du champ de géopotentiel à 200hPa sont-elles les plus visibles ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# Etude de la relation masse-température

L’épaisseur mesure la distance (en mètres) entre 2 niveaux de pression. En météorologie, on étudie traditionnellement l’épaisseur de la couche 1000-500 hPa : dz = z500 - z1000.

In [ ]:
axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle(str(plevz1)+'-'+str(plevz2)+' hPa average temperature (K) and thickness (mgp) : NCEP '+year1+'-'+year2, fontsize=12)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
   ax.set_title(seasons[i], fontsize=10)
   plot_background(ax)
   cf = ax.contourf(lon, lat, tavg[i,:,:], levels_tavg, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
   c = ax.contour(lon, lat, dz[i,:,:], levels_dz, colors='black', linewidths=1, transform=ccrs.PlateCarree())   
   axgr.cbar_axes[i].colorbar(cf)
   ax.clabel(c, inline=1, fmt='%4.1i', fontsize=10)

figname='./figs/thickness_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> D’après les cartes affichées déduire le lien entre l'épaisseur d'une couche et sa température moyenne.</p>
<p><b>2) </b> Quelle est la signature des dépressions thermiques continentales en termes d’épaisseur et de température moyenne de la couche 1000-500 hPa ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# Etude de la relation masse-vent

La règle de Buys-Ballot doit son nom au météorologiste hollandais Buys-Ballot (1817- 1890) ; ce dernier l’avait déduite à son époque des observations météorologiques dont il disposait. Cette règle donne une bonne approximation du vent réel aux moyennes latitudes, à « grande » échelle, sauf très près du sol.

In [ ]:
plevz=200
ilev = list(lev).index(plevz)
levels_z = np.arange(10500,12800,100)

axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Geopotential height (mgp) and winds at '+str(plevz)+' hPa : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    ax.set_title(seasons[i], fontsize=14)
    plot_background(ax)
    cf = ax.contourf(lon, lat, z[i,ilev,:,:], levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    axgr.cbar_axes[i].colorbar(cf)
    c = ax.contour(lon, lat, z[i,ilev,:,:], levels_z, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
    #   wind_slice = slice(None, None, 3)
    #   wind = ax.barbs(lon[wind_slice], lat[wind_slice],
    #                u[i,ilev,:,:][wind_slice, wind_slice], v[i,ilev,:,:][wind_slice, wind_slice],
    #                color='blue', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
    #                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())
    if plevz == 200:
        wind = ax.quiver(lon, lat, u[i,ilev,:,:], v[i,ilev,:,:],
                         scale=1000, angles='xy', regrid_shape=40,
                         transform=ccrs.PlateCarree())
    if plevz == 925:
        wind = ax.quiver(lon, lat, u[i,ilev,:,:], v[i,ilev,:,:],
                         scale=600, angles='xy', regrid_shape=40,
                         transform=ccrs.PlateCarree())    
    if plevz == 850:
        wind = ax.quiver(lon, lat, u[i,ilev,:,:], v[i,ilev,:,:],
                         scale=600, angles='xy', regrid_shape=40,
                         transform=ccrs.PlateCarree())    

figname='./figs/zuv'+str(plevz)+'_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
plevz=850
ilev = list(lev).index(plevz)
levels_z = np.arange(1200,1620,20)

axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Geopotential height (mgp) and winds at '+str(plevz)+' hPa : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    ax.set_title(seasons[i], fontsize=14)
    plot_background(ax)
    cf = ax.contourf(lon, lat, z[i,ilev,:,:], levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    axgr.cbar_axes[i].colorbar(cf)
    c = ax.contour(lon, lat, z[i,ilev,:,:], levels_z, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
    #   wind_slice = slice(None, None, 3)
    #   wind = ax.barbs(lon[wind_slice], lat[wind_slice],
    #                u[i,ilev,:,:][wind_slice, wind_slice], v[i,ilev,:,:][wind_slice, wind_slice],
    #                color='blue', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
    #                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())
    if plevz == 200:
        wind = ax.quiver(lon, lat, u[i,ilev,:,:], v[i,ilev,:,:],
                         scale=1000, angles='xy', regrid_shape=40,
                         transform=ccrs.PlateCarree())
    if plevz == 925:
        wind = ax.quiver(lon, lat, u[i,ilev,:,:], v[i,ilev,:,:],
                         scale=600, angles='xy', regrid_shape=40,
                         transform=ccrs.PlateCarree())    
    if plevz == 850:
        wind = ax.quiver(lon, lat, u[i,ilev,:,:], v[i,ilev,:,:],
                         scale=600, angles='xy', regrid_shape=40,
                         transform=ccrs.PlateCarree())    

figname='./figs/zuv'+str(plevz)+'_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
plevz=925
ilev = list(lev).index(plevz)
levels_z = np.arange(400,910,10)

axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Geopotential height (mgp) and winds at '+str(plevz)+' hPa : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    ax.set_title(seasons[i], fontsize=14)
    plot_background(ax)
    cf = ax.contourf(lon, lat, z[i,ilev,:,:], levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    axgr.cbar_axes[i].colorbar(cf)
    c = ax.contour(lon, lat, z[i,ilev,:,:], levels_z, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
    #   wind_slice = slice(None, None, 3)
    #   wind = ax.barbs(lon[wind_slice], lat[wind_slice],
    #                u[i,ilev,:,:][wind_slice, wind_slice], v[i,ilev,:,:][wind_slice, wind_slice],
    #                color='blue', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
    #                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())
    if plevz == 200:
        wind = ax.quiver(lon, lat, u[i,ilev,:,:], v[i,ilev,:,:],
                         scale=1000, angles='xy', regrid_shape=40,
                         transform=ccrs.PlateCarree())
    if plevz == 925:
        wind = ax.quiver(lon, lat, u[i,ilev,:,:], v[i,ilev,:,:],
                         scale=600, angles='xy', regrid_shape=40,
                         transform=ccrs.PlateCarree())    
    if plevz == 850:
        wind = ax.quiver(lon, lat, u[i,ilev,:,:], v[i,ilev,:,:],
                         scale=600, angles='xy', regrid_shape=40,
                         transform=ccrs.PlateCarree())    

figname='./figs/zuv'+str(plevz)+'_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> Compléter le texte suivant énonçant la règle de Buys-Ballot : A grande échelle dans l’atmosphère, le vent horizontal est presque  ............ aux isohypses. Dans l’hémisphère Nord, il laisse les ............ géopotentiels sur sa gauche et les ............ géopotentiels sur sa droite. L’intensité du vent horizontal est d’autant plus ............ que la pente de la surface isobare est importante.</p>
<p><b>2) </b> Sur quelle face des anticyclones subtropicaux soufflent les alizés (équatoriale ou polaire) ?</p>
<p><b>3) </b> Décrire la circulation moyenne en basses couches et en altitude observée sur l'océan Indien nord et l'Inde en JJA. Comment se nomme ce phénomène ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

<div class="alert alert-danger">
<b>Question : </b>
<p><b>1) </b> En s'inspirant du code ci-dessus, proposer un code permettant le tracé de l'altitude géopotentielle et du module du vent horizontal à 200 hPa.</p>
</div>

# Etude des champs d'humidité et précipitations

L’humidité spécifique désigne la masse de vapeur d’eau contenu dans une unité de masse d’air humide. On exprime usuellement l’humidité spécifique en g/kg (grammes de vapeur d’eau par kilogrammes d’air humide). L’eau précipitable correspond à l’intégrale verticale de l’humidité spécifique. Cette grandeur permet d’avoir une mesure absolue du contenu en eau de l’atmosphère et correspond à la quantité d’eau recueillie si toute la vapeur d’eau d’une colonne d’atmosphère de base d’1 m2 condense. On l’exprime en kg/m² ou millimètres (1 mm = 1 litre d’eau par m²). Dans la troposphère, on trouve aussi de l’eau en phase liquide ou solide en suspension dans les nuages ou dans son trajet vers le sol (précipitations) : hydrométéores. Les précipitations désignent le flux d’eau sous forme liquide ou solide issu de la condensation de la vapeur d’eau dans l’atmosphère.

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Specific humidity - zonal mean : '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    ax.set_yscale('symlog')
    ax.set_yticklabels(np.arange(1000, 300, -100))
    ax.set_ylim(1000, 300)
    ax.set_yticks(np.arange(1000, 300, -100))  
    ax.set_xticklabels(np.arange(-90, 100, 10))
    ax.set_xticks(np.arange(-90, 100, 10))    
    cf = ax.contourf(lat, levq, qz[i,:,:], levels_qz, cmap='jet', extend='both')
    c = ax.contour(lat, levq, qz[i,:,:], levels_qz, colors='black', linewidths=1)
    plt.clabel(c, levels_qz, fmt='%2.1i')
 
cf = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cf.set_label('g/kg', size='small')

fig.set_constrained_layout_pads(w_pad=0., h_pad=0.1, hspace=0., wspace=0.)


figname='./figs/q_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Dans quelle partie de la troposphère et à quelles latitudes se situent principalement les maximas en vapeur d’eau ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

In [ ]:
axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Total Precipitable water (mm) : '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(seasons[i], fontsize=14)
    cf = ax.contourf(lon_pw, lat_pw, pw[i,:,:], levels_pw, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    c = ax.contour(lon_pw, lat_pw, pw[i,:,:], levels_pw, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

figname='./figs/pw_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Selon les saisons, localiser les principales zones de maxima/minima d'eau précipitable sur océan et sur continent.</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

In [ ]:
axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Precipitation (mm/day) : GPCP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.plot([0, 180], [0, 0] , color='k', linewidth=0.5, transform=ccrs.PlateCarree())
    ax.plot([-180, 0], [0, 0] , color='k', linewidth=0.5, transform=ccrs.PlateCarree())
    ax.set_title(seasons[i], fontsize=14)
    cf = ax.contourf(lon_pr, lat_pr, pr[i,:,:], levels_pr, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
    c = ax.contour(lon_pr, lat_pr, pr[i,:,:], levels_pr, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

figname='./figs/precip_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> Selon les saisons, localiser les zones de précipitations les plus intenses.</p>
<p><b>2) </b> Aux moyennes latitudes en quelle saison les précipitations sont-elles les plus importantes ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

In [ ]:
for i in range(12): 
    #print(months[i])
    fig = plt.figure(figsize=(15., 8.))
    fig.suptitle('Precipitation (mm/day) : GPCP '+year1+'-'+year2, fontsize=16)
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    ax.set_title(months[i], fontsize=14)
    plot_background(ax)
    ax.plot([0, 180], [0, 0] , color='k', linewidth=0.5, transform=ccrs.PlateCarree())
    ax.plot([-180, 0], [0, 0] , color='k', linewidth=0.5, transform=ccrs.PlateCarree())
    cf = ax.contourf(lon_pr, lat_pr, pr_month[i,:,:], levels_pr, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
    c = ax.contour(lon_pr, lat_pr, pr_month[i,:,:], levels_pr, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.15,extendrect='True')
    cb.set_label('mm/day', size='small')

    if i<10:
        figname='./anim/precip_monclim_0'+str(i)
    if i>=10:
        figname='./anim/precip_monclim_'+str(i)    
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.show()

In [ ]:
def make_animation():
    nbimages=12
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format('./anim/')))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    os.chdir("./anim/")
    for f in glob.glob("*.png"):
        os.remove(f)
    os.chdir("../")
    return Image

In [ ]:
gif_filepath = './anim/precip_monclim.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

# <span style='background :yellow' > Défi codage 1 (*) : précipitations en moyenne zonale </span>

<div class="alert alert-danger">
Produire un graphique des précipitations en moyenne zonale en moyennes saisonnières (DJF-JJA-MAM-SON) et annuelle. On exploitera le tableau pr(season,lat,lon) qui contient les climatologies saisonnières globales des précipitations GPCP sur la période 1990-2019.
</div>

In [ ]:
print(pr.shape)
lats=pr.lat.values
lons=pr.lon.values
print(lats)
print(lons)

# <span style='background :yellow' > Défi codage 2 (*) : différences de précipitations </span>

<div class="alert alert-danger">
Produire une carte en projection PlateCarree sur un domaine borné entre les latitudes 40S et 40N de la différence des précipitations entre la saison JJA et la saison DJF. Mettre en évidence les différences >6mm/jour et les différences inférieures à -6mm/jour. On exploitera le tableau pr(season,lat,lon) qui contient les climatologies saisonnières globales des précipitations GPCP sur la période 1990-2019. On utilisera la fonction plot_map(ax) pour le tracé du fond de carte.
</div>

In [ ]:
def plot_map(ax):
    bounds = [(0, 357.5, -40, 40)]
    ax.set_xticks(np.linspace(-180, 180, 13), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-40, 40, 5), crs=ccrs.PlateCarree())
    ax.axes.axis('tight')
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.coastlines()
    ax.plot([0, 180], [0, 0] , color='k', linewidth=0.5, transform=ccrs.PlateCarree())
    ax.plot([-180, 0], [0, 0] , color='k', linewidth=0.5, transform=ccrs.PlateCarree())
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    return ax

In [ ]:
print(pr.shape)
lats=pr.lat.values
lons=pr.lon.values
print(lats)
print(lons)

# <span style='background :yellow' > Défi codage 3 (**) : cycle annuel de la ZCIT par secteur </span>

<div class="alert alert-danger">
Produire un graphique similaire à la figure ci-dessous permettant d'étudier le cycle annuel moyen des précipitations dans 8 secteurs de longitudes différents
<img src="zcit.png" width="500"/>

## Indications

On exploitera le Dataset fpr qui contient les séries mensuelles globales des précipitations GPCP sur la période 1990-2019 (variable precip). On utilisera aussi les méthodes groupby, sel et mean pour calculer les climatologies mensuelles et sélectionner les domaines de latitudes et longitudes. Compte tenu que les longitudes sont des 0 à 360 on choisira ici les bornes de latitudes et longitudes suivantes :

- Latitudes : 30S à 30N 

- Afrique : longitudes 10 à 40
- Indien : longitudes 60 à 100
- Pacifique ouest : longitudes 110 à 150
- Pacifique central : longitudes 160 à 200
- Pacifique est : longitudes 220 à 260
- Amerique centrale et du sud : longitudes 285 à 315
- Atlantique : longitudes 320 à 350
    
</div>

In [ ]:
print(fpr)
fpr_africa=fpr.groupby('time.month').mean('time').sel(lat=slice(-30,30)).sel(lon=slice(10,40)).mean('lon')
pr_africa=fpr_africa['precip']
lats=fpr_africa.lat.values
print(pr_africa.shape)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b> Dans quels secteurs la Zone de convergence Inter tropicale (ZCIT) est-elle la plus large ?</p>
<p><b>2) </b> Dans quels bassins océaniques la ZCIT est-elle quasi-exclusivement localisée dans l'hémisphère nord ?</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# <span style='background :yellow' > Défi codage 4 (*) : vent zonal dans la stratosphère équatoriale </span>

<div class="alert alert-danger">
Produire un graphique de l'évolution du vent zonal dans la stratosphère (entre 70hPa et 10hPa) au dessus de l'équateur. On exploitera le Dataset fu qui contient les séries mensuelles globales de vent zonal sur la période 1990-2019 (variale uwnd). On utilisera aussi les méthodes sel et mean pour sélectionner la latitude et effectuer la moyenne zonale.
</div>

In [ ]:
print(fu)

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b> Quel phénomène observe-t-on dans la stratosphère équatoriale ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>